In [14]:
import csv
import json
import sys
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson

user_cookie = dict()

for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

urllist = ["https://lpl.qq.com/es/stats.shtml?bmid=7325",
        "https://lpl.qq.com/es/stats.shtml?bmid=6909"]

test_url = "https://lpl.qq.com/es/stats.shtml?bmid=6909"

#control = {'side':'left', 'count':0, 'records':0}
#json_data['bMatchId']
#jd = json.loads(json_data["sMatchInfo"][0]['battleInfo']['BattleData'])
#jd['left']['players']

def lpl_url_for_request_scraping(raw_url, test):
    base_url = "https://lpl.qq.com/web201612/data/LOL_MATCH_DETAIL_"
    js = ".js"
    match_num = raw_url[-4:]
    new_url = base_url + match_num + js
    if test == True:
        pprint.pprint(new_url)
    return (new_url)

#Get the lpl json data. 
def lpl_get_match_data(url, test):
    good_url = lpl_url_for_request_scraping(url, False)
    json_file = requests.get(good_url)
    json_content = json.loads(json_file.text[12:-1]) # or demjson.decode(json_file.content[12:-1])
    #assert json_file.status_code == 200
    if test == True:
        pprint.pprint(json_content)
    return(json_content)


y = lpl_get_match_data(test_url, False)
print(type(y))

<class 'dict'>


In [9]:
pip install demjson

Using legacy setup.py install for demjson, since package 'wheel' is not installed.
    Running setup.py install for demjson: started
    Running setup.py install for demjson: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.
